# Consultas a la BBDD
En este archivo se prueban consutlas a al BBDD usando lenguaje natural

---

## Setup

### Instalaciones

In [ ]:
# %pip install langgraph --quiet

### Importaciones

In [36]:
from dotenv import load_dotenv
import pandas as pd
from pathlib import Path
import json
from dotenv import load_dotenv
import os 
from IPython.display import Image, display, Markdown
import pprint

import sqlite3

from langchain.chat_models import init_chat_model
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import BaseMessage, HumanMessage

from typing import Literal
from langchain_core.messages import AIMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode


### Variables de entorno

In [3]:
# Load environment variables from .env file
load_dotenv()

True

### preparar BBDD para consultar con langchain

In [ ]:
# This is a SQL wrapper provided by Langchain
# Create the SQLite database from the SQL script if it doesn't exist
db_path = Path("../01_setup/data/consultorio.db")
consultorio_db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# Test the database
print(f"Available tables: {consultorio_db.get_usable_table_names()}")
pprint.pprint(f'Sample output: {consultorio_db.run("SELECT * FROM paciente LIMIT 5;")}')
pprint.pprint(f'Sample output: {consultorio_db.run("SELECT * FROM evolucion LIMIT 5;")}')

### Seleccinar model con el cual interactuar

In [ ]:
# Select an LLM
# llm_model = "openai:"+os.getenv("OPENAI_MODEL")
llm_model = "openai:gpt-4o"
print(llm_model)
llm = init_chat_model(llm_model, temperature=0)

### Preparar el toolkit creando una tool para que el modelo pueda consultar la BBDD

In [ ]:
toolkit = SQLDatabaseToolkit(db=consultorio_db, llm=llm)
tools = toolkit.get_tools()
for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

### Preparando prompt de consulta SQL

In [21]:
SYSTEM_PROMPT = """
Eres un agente diseñado para interactuar con una base de datos SQL.
Dada una pregunta de entrada, crea una consulta {dialect} sintácticamente correcta para ejecutar,
luego revisa los resultados de la consulta y devuelve la respuesta. A menos que el usuario
especifique un número específico de ejemplos que desea obtener, limita siempre tu
consulta a un máximo de {top_k} resultados.

Puedes ordenar los resultados por una columna relevante para devolver los ejemplos más interesantes de la base de datos. Nunca consultes todas las columnas de una tabla específica,
solo solicita las columnas relevantes según la pregunta.

DEBES revisar tu consulta cuidadosamente antes de ejecutarla. Si obtienes un error al
ejecutar una consulta, reescribe la consulta y vuelve a intentarlo.

NO realices ninguna declaración DML (INSERT, UPDATE, DELETE, DROP, etc.) en la
base de datos.

Para comenzar, SIEMPRE debes revisar las tablas en la base de datos para ver qué puedes consultar. NO omitas este paso.

Luego debes consultar el esquema de las tablas más relevantes.
""".format(
    dialect=consultorio_db.dialect,
    top_k=5,
)

agent = create_react_agent(
    llm,
    tools,
    prompt=SYSTEM_PROMPT,
)

---

## Consulta #1
Técnica:
- Usar tool de lanchain y pasarle la tool al modelo para que sepa que puede resolver la consulta disponiendo de las herramientas que le provee el tool
Consulta
- Obtener top 3 de pacientes con mas consultas hechas

In [ ]:
question = "Obtener los tres pacientes que más evoluciones tienen registradas. Como salida necesito el id_paciente, nombre y apellido y número de evoluciones."

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

consulta SQL para comprobar que el resultado anterior es correcto

In [40]:
sql_query = """
SELECT p.id_paciente, p.nombre, p.apellido, COUNT(e.id) AS numero_evoluciones
FROM paciente p
JOIN evolucion e ON p.id_paciente = e.id_paciente
GROUP BY p.id_paciente, p.nombre, p.apellido
ORDER BY numero_evoluciones DESC
LIMIT 5;
"""

In [41]:
def query_to_database(query, db):
    # 1. Connect to the SQLite database
    conn = sqlite3.connect(db)

    try:
        df = pd.read_sql_query(query, conn)
        print(df)
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

query_to_database(sql_query, '../01_setup/data/consultorio.db')  

   id_paciente            nombre      apellido  numero_evoluciones
0          172               Leo      Martínez                   3
1          247           Luciana      Gonzalez                   3
2          280  Matías Alejandro  López Torres                   3
3          131             Lucas      González                   2
4          153          Leonardo      Martinez                   2


---

## Consulta #2